In [1]:
!pip install sentencepiece torch indic-nlp-library
!pip install git+https://github.com/VarunGumma/IndicTransToolkit.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install transformers==4.44.0 --quiet

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IndicTransToolkit import IndicProcessor

In [4]:
from huggingface_hub import login

# Replace with your actual token from https://huggingface.co/settings/tokens
login(token="YOUR_HF_TOKEN_HERE")

In [9]:
model_name = "ai4bharat/indictrans2-en-indic-1B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, trust_remote_code=True
)

# Initialize the IndicProcessor for preprocessing
ip = IndicProcessor(inference=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

...

modeling_indictrans.py:   0%|          | 0.00/79.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

IndicTransForConditionalGeneration(
  (model): IndicTransModel(
    (encoder): IndicTransEncoder(
      (embed_tokens): Embedding(32322, 1024, padding_idx=1)
      (embed_positions): IndicTransSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-17): 18 x IndicTransEncoderLayer(
          (self_attn): IndicTransAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05,

In [10]:
def translate_english_to_marathi(text):
    src_lang = "eng_Latn"
    tgt_lang = "mar_Deva"
    
    # Preprocess the input text
    input_sentences = [text]
    batch = ip.preprocess_batch(input_sentences, src_lang=src_lang, tgt_lang=tgt_lang)
    
    inputs = tokenizer(
        batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    ).to(device)

    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_length=256,
            num_beams=5,
            early_stopping=True,
            num_return_sequences=1
        )

    # Decode the output
    with tokenizer.as_target_tokenizer():
        output_text = tokenizer.batch_decode(
            output_tokens,
            skip_special_tokens=True
        )
    
    # Postprocess the output
    output_text = ip.postprocess_batch(output_text, lang=tgt_lang)
    
    return output_text[0]

In [12]:
english_text = """
Smoking, drinking and drug use during pregnancy can lead to serious health problems for both the mother and the baby. These substances can increase the risk of miscarriage, premature birth, low birth weight, and developmental issues in children. It is important for pregnant women to avoid these substances to ensure a healthy pregnancy and baby.
"""

marathi_translation = translate_english_to_marathi(english_text)

print("English Text:\n", english_text)
print("\nMarathi Translation:\n", marathi_translation)

English Text:
 
Smoking, drinking and drug use during pregnancy can lead to serious health problems for both the mother and the baby. These substances can increase the risk of miscarriage, premature birth, low birth weight, and developmental issues in children. It is important for pregnant women to avoid these substances to ensure a healthy pregnancy and baby.


Marathi Translation:
 गर्भधारणेदरम्यान धूम्रपान, मद्यपान आणि अंमली पदार्थांच्या वापरामुळे आई आणि बाळ दोघांनाही आरोग्याच्या गंभीर समस्या उद्भवू शकतात. या पदार्थांमुळे गर्भपात, अकाली जन्म, जन्माच्या वेळी कमी वजन आणि मुलांमध्ये विकासाच्या समस्यांचा धोका वाढू शकतो. निरोगी गर्भधारणा आणि बाळ सुनिश्चित करण्यासाठी गर्भवती महिलांनी हे पदार्थ टाळणे महत्वाचे आहे.
